In [1]:
from yolo import YOLO # importing yolo

model = YOLO(
    model_path="model_data/yolo3-spp.h5",
    anchors_path="model_data/yolo_anchors.txt",
    classes_path="model_data/coco_classes.txt",
    score=0.3,
    iou=0.45,
    model_image_size=(512, 512),
    gpu_num=1,
)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
model_data/yolo3-spp.h5 model, anchors, and classes loaded.


In [2]:
from yolo3.utils import letterbox_image # importing sth idk
from keras import backend as K # importing keras lib
from timeit import default_timer as timer
class ObjDetector: #object detecting based on the already pretrained model
    def __init__(self): # setting self?
        self.model = YOLO(
            model_path="model_data/yolo3-spp.h5",
            anchors_path="model_data/yolo_anchors.txt",
            classes_path="model_data/coco_classes.txt",
            score=0.3,
            iou=0.45,
            model_image_size=(512, 512),
            gpu_num=1,
        )
    
    def detect(self, image):


        if self.model.model_image_size != (None, None): # converting picture resolution to fit model_image_size
            assert self.model.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')
        
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.model.sess.run(
            [self.model.boxes, self.model.scores, self.model.classes],
            feed_dict={
                self.model.yolo_model.input: image_data,
                self.model.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })
        
        return self.visualize(image, out_boxes, out_scores, out_classes, desired_classes)
        #to return image back to detect_vid
    
    def visualize(self, image, out_boxes, out_scores, out_classes, desired_classes):
        image = np.array(image)
        for i, c in reversed(list(enumerate(out_classes))):
            print(out_classes)
            if desired_classes == c+1 or desired_classes == 0: # 0 to find everything
                predicted_class = self.model.class_names[c]
                box = out_boxes[i]
                score = out_scores[i]

                label = '{} {:.2f}'.format(predicted_class, score)

                top, left, bottom, right = box
                top = max(0, np.floor(top + 0.5).astype('int32'))
                left = max(0, np.floor(left + 0.5).astype('int32'))
                bottom = min(image.shape[0], np.floor(bottom + 0.5).astype('int32'))
                right = min(image.shape[1], np.floor(right + 0.5).astype('int32'))
                print(label, (left, top), (right, bottom))
                
                
                cv2.rectangle(image,(left,top),(right,bottom),(0,255,0),3)
                # remove image showing
        return image # return images for video showing
     
    def detect_vid(self, video_path, detector, output_path=""): # change arguments
        import cv2
        vid = cv2.VideoCapture(video_path)
        if not vid.isOpened():
            raise IOError("Couldn't open webcam or video")
        video_FourCC    = int(vid.get(cv2.CAP_PROP_FOURCC))
        video_fps       = vid.get(cv2.CAP_PROP_FPS)
        video_size      = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
                            int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        isOutput = True if output_path != "" else False
        if isOutput:
            print("!!! TYPE:", type(output_path), type(video_FourCC), type(video_fps), type(video_size))
            out = cv2.VideoWriter(output_path, video_FourCC, video_fps, video_size)
        accum_time = 0
        curr_fps = 0
        fps = "FPS: ??"
        prev_time = timer()
        while True:
            return_value, frame = vid.read()
            image = Image.fromarray(frame)
            
            image = detector.detect(image)
            result = np.asarray(image)
            curr_time = timer()
            exec_time = curr_time - prev_time
            prev_time = curr_time
            accum_time = accum_time + exec_time
            curr_fps = curr_fps + 1
            if accum_time > 1:
                accum_time = accum_time - 1
                fps = "FPS: " + str(curr_fps)
                curr_fps = 0
            cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=0.50, color=(255, 0, 0), thickness=2)
            cv2.namedWindow("result", cv2.WINDOW_NORMAL)
            cv2.imshow("result", result)
            if isOutput:
                out.write(result)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        yolo.close_session()



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

detector = ObjDetector()
path = "./img12.jpg"
image = Image.open(path)


In [3]:
import cv2
desired_classes = 1
results = detector.detect(image)

NameError: name 'detector' is not defined

In [ ]:
""""import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
""""""

In [4]:
from PIL import Image
import numpy as np

video = ObjDetector()


model_data/yolo3-spp.h5 model, anchors, and classes loaded.


In [5]:
import cv2
desired_classes = 0
result_video = video.detect_vid(0, video)

[0 0 0 0]
person 0.47 (0, 394) (44, 421)
[0 0 0 0]
person 0.50 (66, 397) (118, 421)
[0 0 0 0]
person 0.71 (199, 202) (409, 414)
[0 0 0 0]
person 0.79 (460, 344) (554, 420)
[0 0 0 0 0]
person 0.54 (444, 404) (483, 422)
[0 0 0 0 0]
person 0.66 (189, 200) (427, 414)
[0 0 0 0 0]
person 0.68 (69, 401) (119, 422)
[0 0 0 0 0]
person 0.69 (0, 395) (43, 421)
[0 0 0 0 0]
person 0.95 (479, 349) (574, 419)
[0 0 0 0 0]
person 0.58 (445, 404) (482, 421)
[0 0 0 0 0]
person 0.62 (0, 394) (44, 421)
[0 0 0 0 0]
person 0.70 (70, 401) (119, 422)
[0 0 0 0 0]
person 0.77 (186, 200) (426, 411)
[0 0 0 0 0]
person 0.97 (479, 348) (574, 419)
[0 0 0 0]
person 0.62 (0, 395) (44, 421)
[0 0 0 0]
person 0.72 (70, 401) (119, 422)
[0 0 0 0]
person 0.73 (433, 350) (573, 423)
[0 0 0 0]
person 0.75 (186, 196) (421, 413)
[0 0 0 0 0]
person 0.55 (445, 405) (484, 422)
[0 0 0 0 0]
person 0.62 (180, 190) (421, 415)
[0 0 0 0 0]
person 0.65 (0, 395) (44, 421)
[0 0 0 0 0]
person 0.70 (70, 401) (120, 421)
[0 0 0 0 0]
person 0.95 

[0 0 0]
person 0.84 (0, 393) (57, 421)
[0 0 0]
person 0.97 (81, 125) (522, 409)
[0 0 0]
person 0.98 (482, 347) (564, 420)
[0 0 0]
person 0.84 (0, 393) (56, 421)
[0 0 0]
person 0.97 (81, 125) (520, 407)
[0 0 0]
person 0.98 (482, 346) (565, 420)
[0 0 0]
person 0.80 (0, 393) (56, 421)
[0 0 0]
person 0.96 (82, 126) (522, 407)
[0 0 0]
person 0.99 (482, 346) (565, 420)
[0 0 0]
person 0.87 (0, 394) (55, 420)
[0 0 0]
person 0.95 (83, 127) (522, 410)
[0 0 0]
person 0.99 (483, 345) (566, 420)
[0 0 0]
person 0.85 (0, 393) (55, 420)
[0 0 0]
person 0.95 (81, 124) (526, 407)
[0 0 0]
person 0.99 (486, 346) (567, 419)
[0 0 0]
person 0.85 (0, 394) (53, 420)
[0 0 0]
person 0.97 (84, 123) (520, 407)
[0 0 0]
person 0.99 (486, 347) (571, 418)
[0 0 0]
person 0.84 (0, 393) (54, 421)
[0 0 0]
person 0.96 (82, 117) (523, 409)
[0 0 0]
person 0.99 (486, 347) (573, 419)
[0 0 0]
person 0.85 (1, 393) (54, 420)
[0 0 0]
person 0.96 (83, 119) (520, 410)
[0 0 0]
person 0.99 (487, 348) (574, 418)
[0 0 0]
person 0.81 (0, 

[0 0 0 0]
person 0.36 (53, 400) (103, 418)
[0 0 0 0]
person 0.79 (0, 394) (53, 419)
[0 0 0 0]
person 0.96 (83, 128) (524, 407)
[0 0 0 0]
person 0.99 (474, 346) (557, 418)
[0 0 0 0]
person 0.41 (52, 400) (102, 418)
[0 0 0 0]
person 0.78 (0, 394) (53, 419)
[0 0 0 0]
person 0.94 (83, 126) (524, 408)
[0 0 0 0]
person 0.99 (475, 346) (558, 418)
[0 0 0 0]
person 0.47 (52, 399) (102, 418)
[0 0 0 0]
person 0.80 (0, 394) (53, 419)
[0 0 0 0]
person 0.94 (80, 124) (526, 408)
[0 0 0 0]
person 0.96 (480, 345) (564, 419)
[0 0 0 0]
person 0.48 (52, 400) (102, 418)
[0 0 0 0]
person 0.85 (0, 394) (53, 419)
[0 0 0 0]
person 0.94 (84, 125) (524, 408)
[0 0 0 0]
person 0.99 (483, 344) (567, 420)
[0 0 0 0]
person 0.46 (52, 400) (102, 418)
[0 0 0 0]
person 0.83 (0, 394) (53, 419)
[0 0 0 0]
person 0.93 (80, 125) (527, 408)
[0 0 0 0]
person 0.99 (482, 345) (570, 419)
[0 0 0 0]
person 0.42 (52, 399) (102, 419)
[0 0 0 0]
person 0.81 (1, 394) (53, 419)
[0 0 0 0]
person 0.94 (83, 125) (524, 407)
[0 0 0 0]
person 0

[0 0 0 0]
person 0.67 (52, 402) (105, 421)
[0 0 0 0]
person 0.69 (0, 395) (50, 420)
[0 0 0 0]
person 0.93 (103, 136) (560, 402)
[0 0 0 0]
person 0.99 (491, 350) (577, 419)
[0 0 0 0]
person 0.63 (0, 396) (49, 420)
[0 0 0 0]
person 0.70 (52, 402) (104, 421)
[0 0 0 0]
person 0.92 (104, 131) (565, 406)
[0 0 0 0]
person 0.98 (491, 351) (577, 419)
[0 0 0 0]
person 0.68 (0, 396) (50, 419)
[0 0 0 0]
person 0.68 (52, 402) (102, 421)
[0 0 0 0]
person 0.90 (101, 129) (568, 405)
[0 0 0 0]
person 0.98 (490, 350) (577, 419)
[0 0 0 0]
person 0.61 (49, 398) (101, 419)
[0 0 0 0]
person 0.69 (0, 393) (51, 421)
[0 0 0 0]
person 0.90 (102, 128) (565, 401)
[0 0 0 0]
person 0.99 (491, 346) (578, 419)
[0 0 0 0]
person 0.70 (48, 398) (101, 420)
[0 0 0 0]
person 0.76 (0, 393) (51, 420)
[0 0 0 0]
person 0.89 (101, 129) (566, 403)
[0 0 0 0]
person 0.99 (490, 347) (579, 419)
[0 0 0 0]
person 0.73 (50, 399) (102, 419)
[0 0 0 0]
person 0.80 (0, 394) (51, 421)
[0 0 0 0]
person 0.90 (88, 132) (527, 406)
[0 0 0 0]
per

[0 0 0 0]
person 0.62 (44, 398) (100, 420)
[0 0 0 0]
person 0.64 (0, 394) (44, 422)
[0 0 0 0]
person 0.82 (92, 125) (575, 410)
[0 0 0 0]
person 0.98 (485, 345) (571, 420)
[0 0 0 0]
person 0.64 (0, 394) (44, 421)
[0 0 0 0]
person 0.64 (45, 397) (99, 421)
[0 0 0 0]
person 0.86 (99, 125) (569, 407)
[0 0 0 0]
person 0.98 (486, 344) (572, 420)
[0 0 0 0]
person 0.45 (46, 398) (98, 420)
[0 0 0 0]
person 0.68 (0, 394) (45, 420)
[0 0 0 0]
person 0.92 (88, 132) (525, 407)
[0 0 0 0]
person 0.99 (485, 345) (575, 420)
[0 0 0]
person 0.37 (0, 394) (46, 420)
[0 0 0]
person 0.85 (87, 138) (520, 405)
[0 0 0]
person 0.98 (482, 345) (574, 420)
[0 0 0 0]
person 0.33 (47, 398) (98, 420)
[0 0 0 0]
person 0.52 (0, 394) (46, 420)
[0 0 0 0]
person 0.80 (84, 138) (516, 403)
[0 0 0 0]
person 0.96 (480, 345) (574, 419)
[0 0 0]
person 0.42 (0, 394) (46, 421)
[0 0 0]
person 0.64 (81, 143) (522, 404)
[0 0 0]
person 0.97 (482, 344) (575, 419)
[0 0 0 0]
person 0.46 (47, 398) (99, 420)
[0 0 0 0]
person 0.55 (0, 394) (4

...

[0 0 0 0 0]
person 0.31 (552, 229) (640, 430)
[0 0 0 0 0]
person 0.60 (397, 271) (440, 312)
[0 0 0 0 0]
person 0.78 (428, 214) (536, 421)
[0 0 0 0 0]
person 0.99 (0, 254) (94, 409)
[0 0 0 0 0]
person 1.00 (33, 68) (480, 422)
[ 0  0  0  0  0 56]
chair 0.42 (520, 337) (573, 418)
[ 0  0  0  0  0 56]
person 0.36 (552, 229) (640, 430)
[ 0  0  0  0  0 56]
person 0.77 (400, 269) (444, 305)
[ 0  0  0  0  0 56]
person 0.88 (430, 211) (535, 423)
[ 0  0  0  0  0 56]
person 0.99 (0, 257) (100, 408)
[ 0  0  0  0  0 56]
person 0.99 (48, 68) (494, 423)
[0 0 0 0]
person 0.78 (399, 269) (439, 300)
[0 0 0 0]
person 0.86 (432, 211) (535, 423)
[0 0 0 0]
person 0.97 (0, 257) (106, 409)
[0 0 0 0]
person 0.99 (80, 68) (494, 423)
[ 0  0  0  0 56]
chair 0.38 (520, 336) (574, 419)
[ 0  0  0  0 56]
person 0.76 (428, 213) (536, 422)
[ 0  0  0  0 56]
person 0.85 (401, 267) (442, 304)
[ 0  0  0  0 56]
person 0.97 (0, 257) (105, 411)
[ 0  0  0  0 56]
person 0.99 (83, 68) (490, 424)
[0 0 0 0]
person 0.74 (430, 210) (

[ 0  0  0  0 63]
laptop 0.34 (523, 340) (614, 424)
[ 0  0  0  0 63]
person 0.58 (397, 269) (446, 309)
[ 0  0  0  0 63]
person 0.91 (2, 247) (88, 378)
[ 0  0  0  0 63]
person 0.93 (436, 213) (534, 422)
[ 0  0  0  0 63]
person 0.98 (10, 72) (487, 419)
[ 0  0  0  0 63]
laptop 0.30 (523, 340) (613, 424)
[ 0  0  0  0 63]
person 0.70 (397, 269) (446, 308)
[ 0  0  0  0 63]
person 0.91 (0, 246) (88, 376)
[ 0  0  0  0 63]
person 0.92 (436, 212) (533, 422)
[ 0  0  0  0 63]
person 0.98 (8, 70) (489, 420)
[0 0 0 0]
person 0.69 (395, 270) (442, 304)
[0 0 0 0]
person 0.91 (436, 213) (533, 422)
[0 0 0 0]
person 0.96 (0, 249) (90, 377)
[0 0 0 0]
person 0.99 (11, 74) (490, 417)
[ 0  0  0  0 63]
laptop 0.30 (1, 342) (44, 408)
[ 0  0  0  0 63]
person 0.55 (396, 270) (442, 303)
[ 0  0  0  0 63]
person 0.71 (435, 215) (534, 418)
[ 0  0  0  0 63]
person 0.92 (0, 251) (94, 380)
[ 0  0  0  0 63]
person 0.99 (17, 73) (491, 418)
[0 0 0 0]
person 0.45 (399, 270) (445, 312)
[0 0 0 0]
person 0.71 (1, 251) (94, 369

[ 0  0  0  0  0 56]
chair 0.31 (524, 343) (575, 421)
[ 0  0  0  0  0 56]
person 0.34 (403, 272) (433, 301)
[ 0  0  0  0  0 56]
person 0.92 (537, 79) (638, 429)
[ 0  0  0  0  0 56]
person 0.96 (406, 218) (536, 414)
[ 0  0  0  0  0 56]
person 0.99 (82, 65) (501, 425)
[ 0  0  0  0  0 56]
person 1.00 (21, 100) (240, 420)
[0 0 0 0 0 0]
person 0.31 (402, 272) (432, 300)
[0 0 0 0 0 0]
person 0.49 (0, 255) (70, 413)
[0 0 0 0 0 0]
person 0.90 (539, 78) (638, 429)
[0 0 0 0 0 0]
person 0.95 (407, 217) (534, 416)
[0 0 0 0 0 0]
person 0.99 (37, 124) (236, 421)
[0 0 0 0 0 0]
person 0.99 (84, 64) (499, 426)
[ 0  0  0  0  0  0 56]
chair 0.33 (524, 343) (575, 420)
[ 0  0  0  0  0  0 56]
person 0.30 (403, 272) (432, 299)
[ 0  0  0  0  0  0 56]
person 0.87 (5, 256) (89, 416)
[ 0  0  0  0  0  0 56]
person 0.92 (537, 76) (638, 428)
[ 0  0  0  0  0  0 56]
person 0.95 (406, 218) (533, 415)
[ 0  0  0  0  0  0 56]
person 0.99 (44, 127) (230, 417)
[ 0  0  0  0  0  0 56]
person 1.00 (89, 62) (494, 428)
[ 0  0  0

[0 0 0 0 0]
person 0.84 (403, 271) (445, 303)
[0 0 0 0 0]
person 0.93 (425, 217) (545, 418)
[0 0 0 0 0]
person 0.96 (0, 249) (109, 418)
[0 0 0 0 0]
person 0.98 (80, 178) (227, 336)
[0 0 0 0 0]
person 0.99 (63, 70) (484, 421)
[ 0  0  0  0  0  0 56]
chair 0.38 (525, 338) (576, 418)
[ 0  0  0  0  0  0 56]
person 0.40 (551, 311) (639, 424)
[ 0  0  0  0  0  0 56]
person 0.71 (403, 272) (446, 305)
[ 0  0  0  0  0  0 56]
person 0.90 (426, 219) (545, 416)
[ 0  0  0  0  0  0 56]
person 0.98 (0, 252) (108, 416)
[ 0  0  0  0  0  0 56]
person 0.99 (80, 183) (227, 335)
[ 0  0  0  0  0  0 56]
person 0.99 (61, 74) (487, 418)
[0 0 0 0 0 0]
person 0.43 (558, 311) (638, 422)
[0 0 0 0 0 0]
person 0.67 (403, 273) (446, 304)
[0 0 0 0 0 0]
person 0.86 (436, 217) (542, 415)
[0 0 0 0 0 0]
person 0.93 (0, 255) (106, 412)
[0 0 0 0 0 0]
person 0.99 (81, 180) (224, 340)
[0 0 0 0 0 0]
person 0.99 (74, 72) (500, 421)
[ 0  0  0  0  0  0 56]
chair 0.36 (519, 339) (596, 423)
[ 0  0  0  0  0  0 56]
person 0.56 (576, 31

KeyboardInterrupt: 